# TABCMD and Python Report Generator Guide
Block by block walkthrough of the python_start.py file

# Import Required Libraries

In [2]:
import teradata
import pandas as pd
import os
import datetime
import getpass
import glob
import warnings
warnings.filterwarnings("ignore")


# Provide Selection for Example Dashboard
This is the variable that will be utilized for the "thing" parameter on the example dashboard. The other two are selected via SQL Query.


In [3]:
thing='thing start'
thing_str = thing.replace(' ', '%20')

# Enter Credentials for Teradata and Tableau Login
This is optional and can be either embedded or called by storing environment variables

In [4]:
user= input('Enter Username: ')
passw = getpass.getpass('Enter Pass: ')
auth = input('Ether Auth Type (TD2/LDAP): ')

Enter Username: grumpy_statistician
Enter Pass: ········
Ether Auth Type (TD2/LDAP): TD2


# Connect to Teradata

In [10]:
udaExec = teradata.UdaExec(appName="test", version="2.0", logConsole=False)
session = udaExec.connect(method="odbc", system='server_name',
                          username=user, password=passw, authentication=auth,
                          driver="Teradata Database ODBC Driver")

# Get Eom_dt for Reporting Period Filter

In [11]:
eom_df = pd.read_sql("""
SELECT
date_thing
FROM YOUR_DB.YOUR_TABLE;
""",session)
eom_list = list(eom_df.eom_dt)
eom_str = ''.join(eom_list)

# Get Account Names for Account Name Filter
I selected four things for this example. I trimmed, replaced spaces with %20, and added double quotes for each account.
All spaces and special characters must be handled prior to sending to TABCMD as they are required to be URL encoded to function properly. 
<p>To avoid this issue in the future, I would suggest using the thing number field as a filter, rather than the name as it requires no special characters and the characters will not change.</p>

In [12]:
acct_df = pd.read_sql("""
SELECT  
'"'||oreplace(trim(THING_NM),' ','%20')||'"' as Thing_Names
FROM YOUR_DB.YOUR_TABLE
GROUP BY 1
""",session)

# Handle Additional Special Characters for URL Encoding

In [19]:
acct_list = list(acct_df.Account_Names)

In [20]:
#https://www.geeksforgeeks.org/python-replace-substrings-from-string-list/
replace_dict = {',':'%5C%2C','&':'%26'} #handle special chars

for key, val in replace_dict.items():
    for idx, ele in enumerate(acct_list):
        if key in ele:
            thing_list[idx] = ele.replace(key,val)
            
thing2_str=' '.join(thing_list) # use space as delimiter

# Store Variables for Use in the TABCMD Batch Script
Now that we have the variables required for each filter, we need to send them our TABCMD script. We will then call them to use for Tableau login and to feed our URL links.

In [21]:
os.putenv("user",user)
os.putenv("eom_dt", eom_str)
os.putenv("passw", passw)
os.putenv("thing2", thing2_str)
os.putenv("thing1", thing1_str)

# Create folder to Store Outputs

Prior to sending these variables, we need to create a folder to store them in for tracking purposes and to keep our future reporting tidy.

In [23]:
# Create date variables
now = datetime.datetime.now()
month_name = now.strftime("%B")
month_short = month_name[:3]
year = now.year

# Make folder
work_dir = f'{month_name} {year}'
# Set directory (update this for new PY)
parent_dir = os.getcwd()
# Join to main path
path = os.path.join(parent_dir,work_dir)

# Create folder
try:
    os.mkdir(path)
except OSError as error:
    print(error)


# Send path to TABCMD Batch File and Run 

Finally we can send the path to the TABCMD script, so that it knows where the final PDFs should be saved. Final step is to run the file!

In [ ]:
# Send path to batch
os.putenv("path_nm", path)

# Run batch file
os.system("combine.bat")